### Functions

In [10]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Thesis/probabilistic_harmony_model

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Thesis/probabilistic_harmony_model


In [11]:
!pip install pyro-api==0.1.2
!pip install pyro-ppl==1.8.4
!pip install pitchtypes==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 kB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [12]:
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist

from pyro.distributions import *
#from collections import Counter
import pyro.infer
import pyro.optim
import pyro.util
pyro.enable_validation(True)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tqdm

import numpy as np
import pandas as pd
import scipy.stats as stats

import os.path as path
from datetime import datetime
from fractions import Fraction
import json

import utils

import gc

In [13]:
gpu = torch.cuda.is_available()

# TODO: set the GPU you want to use
gpu_n = 0

torch.set_default_dtype(torch.float64)
device = torch.device(f'cuda:{gpu_n}' if gpu else 'cpu')
torch.set_default_device(device)
print(device)

cpu


In [14]:
def save_rng_state(name):
    fn = name + '-' + datetime.today().isoformat() + '.state'
    state = pyro.util.get_rng_state()
    with open('rng-' + fn, 'w') as f:
        print(state, file=f)
    torch.save(state['torch'], 'torch-' + fn)

In [15]:
# set random seeds
pyro.set_rng_seed(0)
#torch.set_deterministic(True)
#torch.set_num_threads(1)
#torch.set_num_interop_threads(1)

# fix the range of pitches we consider
fifth_range = 2*7                  # 2 diatonics
npcs = 2*fifth_range+1             # around C: Cbb to C## on LoF
utils.set_fifth_range(fifth_range) # used to make helper functions work correctly

In [17]:
def chord_tensor(notes):
    """Takes a list of notes as (fifth, type) pairs and returns a vector of counts."""
    notetype = {'chordtone': 0, 'ornament': 1, 'unknown': 2}
    chord = torch.zeros((3, npcs), device=device)
    for (fifth, t) in notes:
        chord[notetype[t], utils.fifth_to_index(fifth)] += 1
    return chord

# def annot_data_obs(chords):
#     """Helper function to turn a list of chord dictionaries into a dictionary of observation vectors."""
#     obs = {}
#     obs["pitches"] = torch.cat([chord_tensor(c['notes']).reshape((1,-1)) for c in chords], dim=0)
#     obs["c"] = torch.tensor([c['label'] for c in chords], dtype=torch.int64, device=device)
#     obs["n"] = torch.tensor([len(c['notes']) for c in chords], dtype=torch.int64, device=device)
#     obs["onset"] = torch.tensor([float(Fraction(c['onset'])) for c in chords], dtype=torch.float32, device=device)
#     obs["filename"] = [c['filename'] for c in chords]
#     return obs

def annot_data_obs(chords):
    """Helper function to turn a list of chord dictionaries into a dictionary of observation vectors, now including chordid."""
    obs = {}
    obs["pitches"] = torch.cat([chord_tensor(c['notes']).reshape((1,-1)) for c in chords], dim=0)
    obs["c"] = torch.tensor([c['label'] for c in chords], device=device)
    obs["n"] = torch.tensor([len(c['notes']) for c in chords], device=device)  # corrected from len - 1 to len
    obs["chordid"] = torch.tensor([c['chordid'] for c in chords], device=device)  # Add chordid to observations
    return obs

In [18]:
def load_dataset(filename):
    filename = path.join("data", filename)
    print("loading dataset...")
    df = utils.load_csv(filename)
    sizes = df.groupby(['chordid', 'label']).size()
    type_counts = sizes.groupby('label').size().sort_values(ascending=False)
    chordtypes = type_counts.index.tolist()
    df['numlabel'] = df.label.map(chordtypes.index)

    prefn = filename + "ori_precomp.pt"
    if path.exists(prefn) and path.getmtime(prefn) > path.getmtime(filename):
        print("using precomputed tensor data.")
        obs = torch.load(prefn, map_location=device)
    else:
        print('extracting chords...')
        chords = [{'chordid': idx, 'label': label, 'notes': list(zip(grp.fifth, grp.type))}
                  for (idx, label), grp in tqdm.tqdm(df.groupby(['chordid', 'numlabel']))]
        print('converting chords to tensors...')
        obs = annot_data_obs(chords)
        torch.save(obs, prefn)

    print(len(chordtypes), "chord types")
    print(len(obs["c"]), "chords")
    return df, obs, chordtypes

In [16]:
def lookup_original_instances(predictions, obs, df):
    chordid_to_cluster = dict(zip(obs["chordid"].cpu().numpy(), predictions+1))
    df["predicted_cluster"] = df["chordid"].map(chordid_to_cluster)
    return df[df["predicted_cluster"].notnull()]

In [19]:
dcml_df, dcml_obs, dcml_chordtypes = load_dataset('dcml2.tsv')

loading dataset...
using precomputed tensor data.
14 chord types
113771 chords


### Table

In [21]:
params = torch.load('dcml_params.pt')
preds_stage2 = np.load('preds_stage2.npy')

dcml_with_clusters = lookup_original_instances(preds_stage2, dcml_obs, dcml_df)
dcml_with_clusters

,chordid,label,fifth,type,onset,filename,numlabel,predicted_cluster
0,0,M,0,chordtone,0,data/dcml_corpora/ABC n01op18-1_01,0,14
1,0,M,0,chordtone,0,data/dcml_corpora/ABC n01op18-1_01,0,14
2,0,M,0,chordtone,0,data/dcml_corpora/ABC n01op18-1_01,0,14
3,0,M,0,chordtone,0,data/dcml_corpora/ABC n01op18-1_01,0,14
4,0,M,2,ornament,0,data/dcml_corpora/ABC n01op18-1_01,0,14
...,...,...,...,...,...,...,...,...
864604,113770,M,4,chordtone,126,data/romantic_piano_corpus/tchaikovsky_seasons...,0,14
864605,113770,M,1,chordtone,126,data/romantic_piano_corpus/tchaikovsky_seasons...,0,14
864606,113770,M,0,chordtone,126,data/romantic_piano_corpus/tchaikovsky_seasons...,0,14
864607,113770,M,4,chordtone,126,data/romantic_piano_corpus/tchaikovsky_seasons...,0,14


from matplotlib import pyplot as plt
dcml_with_clusters['chordid'].plot(kind='hist', bins=20, title='chordid')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
dcml_with_clusters['fifth'].plot(kind='hist', bins=20, title='fifth')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
dcml_with_clusters['numlabel'].plot(kind='hist', bins=20, title='numlabel')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
dcml_with_clusters['predicted_cluster'].plot(kind='hist', bins=20, title='predicted_cluster')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(dcml_with_clusters['type'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(dcml_with_clusters, x='fifth', y='type', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(dcml_with_clusters['type'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(dcml_with_clusters, x='numlabel', y='type', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(dcml_with_clusters['type'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(dcml_with_clusters, x='predicted_cluster', y='type', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)